In [19]:
# importing libraries

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.agents import *
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor
from langchain import OpenAI, SQLDatabaseChain
from langchain import PromptTemplate, LLMChain
from langchain import HuggingFacePipeline
import sqlalchemy as sqla
from sqlalchemy import text
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import huggingface_hub
from langchain import HuggingFaceHub
from transformers import pipeline



In [21]:
# connect to datatbase
load_dotenv()


# Access environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_NAME2 = os.getenv("DB_NAME2")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
HF_KEY= os.getenv("HUGGINGFACEHUB_API_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from huggingface_hub import login
login(token=HF_KEY)


db = SQLDatabase.from_uri(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
db2 = SQLDatabase.from_uri(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME2}")
engine = sqla.create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chelciedealmeida/.cache/huggingface/token
Login successful


/usr/local/lib/python3.9/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Did not recognize type 'point' of column 'coordinates'
  self._metadata.reflect(


In [22]:
# Load model directly

# llm = ChatOpenAI(model_name="gpt-3.5-turbo")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

toolkit = SQLDatabaseToolkit(db=db2, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)



In [23]:
agent_executor.run("describe the bookings schema?")




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: aircrafts_data, airports_data, boarding_passes, bookings, flights, seats, ticket_flights, tickets
Thought:The relevant table for the bookings schema is "bookings". I should query the schema of this table.
Action: sql_db_schema
Action Input: "bookings"
Observation: 
CREATE TABLE bookings (
	book_ref CHAR(6) NOT NULL, 
	book_date TIMESTAMP WITH TIME ZONE NOT NULL, 
	total_amount NUMERIC(10, 2) NOT NULL, 
	CONSTRAINT bookings_pkey PRIMARY KEY (book_ref)
)

/*
3 rows from bookings table:
book_ref	book_date	total_amount
00000F	2017-07-04 17:12:00-07:00	265700.00
000012	2017-07-13 23:02:00-07:00	37900.00
00002D	2017-05-20 08:45:00-07:00	114700.00
*/
Thought:The bookings schema consists of the following columns:
- book_ref (CHAR(6))
- book_date (TIMESTAMP WITH TIME ZONE)
- total_amount (NUMERIC(10, 2))

The table contains 3 rows with the following data:
- book_ref: 00000F, book_date: 2017-07-04 17

'The bookings schema consists of the columns book_ref, book_date, and total_amount.'

In [24]:
agent_executor.run("describe the flights table?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: aircrafts_data, airports_data, boarding_passes, bookings, flights, seats, ticket_flights, tickets
Thought:The "flights" table is available for querying.
Action: sql_db_schema
Action Input: "flights"
Observation: 
CREATE TABLE flights (
	flight_id SERIAL NOT NULL, 
	flight_no CHAR(6) NOT NULL, 
	scheduled_departure TIMESTAMP WITH TIME ZONE NOT NULL, 
	scheduled_arrival TIMESTAMP WITH TIME ZONE NOT NULL, 
	departure_airport CHAR(3) NOT NULL, 
	arrival_airport CHAR(3) NOT NULL, 
	status VARCHAR(20) NOT NULL, 
	aircraft_code CHAR(3) NOT NULL, 
	actual_departure TIMESTAMP WITH TIME ZONE, 
	actual_arrival TIMESTAMP WITH TIME ZONE, 
	CONSTRAINT flights_pkey PRIMARY KEY (flight_id), 
	CONSTRAINT flights_aircraft_code_fkey FOREIGN KEY(aircraft_code) REFERENCES aircrafts_data (aircraft_code), 
	CONSTRAINT flights_arrival_airport_fkey FOREIGN KEY(arrival_airport) REFERENCES airports_data (airport_code

'The flights table has the following columns: flight_id, flight_no, scheduled_departure, scheduled_arrival, departure_airport, arrival_airport, status, aircraft_code, actual_departure, and actual_arrival.'

In [25]:
agent_executor.run("How many rows are in the bookings table?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: aircrafts_data, airports_data, boarding_passes, bookings, flights, seats, ticket_flights, tickets
Thought:I can query the "bookings" table to get the number of rows.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM bookings"
Observation: [(593433,)]
Thought:I now know the final answer.
Final Answer: There are 593,433 rows in the bookings table.

> Finished chain.


'There are 593,433 rows in the bookings table.'

In [26]:
agent_executor.run("How many rows are in the aircrafts_data table?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: aircrafts_data, airports_data, boarding_passes, bookings, flights, seats, ticket_flights, tickets
Thought:I can query the aircrafts_data table to count the number of rows.
Action: sql_db_query
Action Input: "SELECT COUNT(*) FROM aircrafts_data"
Observation: [(9,)]
Thought:I now know the final answer.
Final Answer: There are 9 rows in the aircrafts_data table.

> Finished chain.


'There are 9 rows in the aircrafts_data table.'